In [1]:
import pandas as pd
import requests
import psycopg2
from sqlalchemy import create_engine

In [2]:
url = 'https://www.goodcarbadcar.net/2021-us-vehicle-sales-figures-by-model/'
# Retrieve page with the requests module
html = requests.get(url)

In [3]:
tables = pd.read_html(html.text)
# capture table data in a dataframe
car_sales = tables[0]
# evaluate the data
car_sales.count()

Model         115
Month         116
Month LY      116
Change        114
YTD           116
YTD LY        116
YTD Change    114
dtype: int64

In [4]:
# drop rows that don't have useful attributes
car_sales.dropna(subset = ['Month', 'Month LY', 'Change', 'YTD Change'], inplace=True)
car_sales.count()

Model         114
Month         114
Month LY      114
Change        114
YTD           114
YTD LY        114
YTD Change    114
dtype: int64

In [5]:
# lets see what we're working with...
car_sales

,Model,Month,Month LY,Change,YTD,YTD LY,YTD Change
0,Mazda 3,2463,3175,-22.43,32216,22634,42.33
1,Toyota 4Runner,13210,9860,33.98,94802,76598,23.77
2,Mazda 6,1435,1230,16.67,15529,10514,47.70
3,Volvo 60-Series,1153,1790,-35.59,9316,8975,3.80
4,Toyota 86 / FR-S,1,236,-99.58,1042,1816,-42.62
...,...,...,...,...,...,...,...
109,Toyota Venza,5329,43,12293.02,46269,43,107502.33
110,Volvo XC40,2640,1933,36.58,20019,13669,46.46
111,Volvo XC60,3266,2981,9.56,28363,17830,59.07
112,Volvo XC90,3542,2998,18.15,27211,19863,36.99


In [6]:
sales_df = pd.DataFrame(car_sales)

In [7]:
# split 1st column into make and model
sales_df[['Make','Model']] = sales_df['Model'].str.split(' ', 1, expand=True)
sales_df

,Model,Month,Month LY,Change,YTD,YTD LY,YTD Change,Make
0,3,2463,3175,-22.43,32216,22634,42.33,Mazda
1,4Runner,13210,9860,33.98,94802,76598,23.77,Toyota
2,6,1435,1230,16.67,15529,10514,47.70,Mazda
3,60-Series,1153,1790,-35.59,9316,8975,3.80,Volvo
4,86 / FR-S,1,236,-99.58,1042,1816,-42.62,Toyota
...,...,...,...,...,...,...,...,...
109,Venza,5329,43,12293.02,46269,43,107502.33,Toyota
110,XC40,2640,1933,36.58,20019,13669,46.46,Volvo
111,XC60,3266,2981,9.56,28363,17830,59.07,Volvo
112,XC90,3542,2998,18.15,27211,19863,36.99,Volvo


In [8]:
# put columns in correct order for loading to the DB
make = sales_df.pop('Make')
sales_df

,Model,Month,Month LY,Change,YTD,YTD LY,YTD Change
0,3,2463,3175,-22.43,32216,22634,42.33
1,4Runner,13210,9860,33.98,94802,76598,23.77
2,6,1435,1230,16.67,15529,10514,47.70
3,60-Series,1153,1790,-35.59,9316,8975,3.80
4,86 / FR-S,1,236,-99.58,1042,1816,-42.62
...,...,...,...,...,...,...,...
109,Venza,5329,43,12293.02,46269,43,107502.33
110,XC40,2640,1933,36.58,20019,13669,46.46
111,XC60,3266,2981,9.56,28363,17830,59.07
112,XC90,3542,2998,18.15,27211,19863,36.99


In [9]:
sales_df.insert(0, 'Make', make)
sales_df

,Make,Model,Month,Month LY,Change,YTD,YTD LY,YTD Change
0,Mazda,3,2463,3175,-22.43,32216,22634,42.33
1,Toyota,4Runner,13210,9860,33.98,94802,76598,23.77
2,Mazda,6,1435,1230,16.67,15529,10514,47.70
3,Volvo,60-Series,1153,1790,-35.59,9316,8975,3.80
4,Toyota,86 / FR-S,1,236,-99.58,1042,1816,-42.62
...,...,...,...,...,...,...,...,...
109,Toyota,Venza,5329,43,12293.02,46269,43,107502.33
110,Volvo,XC40,2640,1933,36.58,20019,13669,46.46
111,Volvo,XC60,3266,2981,9.56,28363,17830,59.07
112,Volvo,XC90,3542,2998,18.15,27211,19863,36.99


In [10]:
sales_df.dropna(subset = ['Make', 'Model'], inplace=True)
sales_df.count()

Make          114
Model         114
Month         114
Month LY      114
Change        114
YTD           114
YTD LY        114
YTD Change    114
dtype: int64

In [11]:
makes_df = sales_df[['Make']].copy()
# sort the data and remove dupes
makes_df.sort_values(by='Make')
makes_df.drop_duplicates(keep='first', inplace=True)
makes_df.reset_index(drop=True, inplace=True)
# rename columns to match SQL table columns
makes_df.rename(columns={"Make": "mfg"}, inplace=True)
makes_df

,mfg
0,Mazda
1,Toyota
2,Volvo
3,Hyundai
4,Honda
5,Subaru
6,Lincoln
7,Ford
8,Kia
9,Lexus


In [12]:
models_df = sales_df[['Make', 'Model']].copy()
# sort the data and remove dupes
models_df.sort_values(by=['Make', 'Model'], inplace=True)
models_df.drop_duplicates(keep='first', inplace=True)
models_df.reset_index(drop=True, inplace=True)
# rename columns to match SQL table columns
models_df.rename(columns={"Make": "mfg", "Model": "model"}, inplace=True)
models_df

,mfg,model
0,Acura,ILX
1,Acura,MDX
2,Acura,NSX
3,Acura,RDX
4,Acura,RLX
...,...,...
109,Volvo,60-Series
110,Volvo,90-Series
111,Volvo,XC40
112,Volvo,XC60


### Connect to local database

In [13]:
from config import pg_user
from config import pg_key

connection_string = "{username}:{password}@localhost:5432/car_sales_2021"
engine = create_engine(f'postgresql://{connection_string}'.format(username=pg_user, password=pg_key))

### Check for tables

In [14]:
engine.table_names()

<ipython-input-14-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['y2021_makes', 'y2021_models', 'y2021_sales']

In [15]:
# Use pandas to load sales DataFrame into database
# first rename columns to match SQL table 
sales_df.rename(columns={"Make": "mfg", "Model": "model", "Month": "curr_month", "Month LY": "month_prior_year",\
                         "Change": "month_change", "YTD": "ytd_total", "YTD LY": "ytd_prior_year",\
                         "YTD Change": "ytd_change"}, inplace=True)
sales_df.count()

mfg                 114
model               114
curr_month          114
month_prior_year    114
month_change        114
ytd_total           114
ytd_prior_year      114
ytd_change          114
dtype: int64

In [16]:
### now load data into database
sales_df.to_sql(name='y2021_sales', con=engine, if_exists='replace', index=False)

In [17]:
# Confirm data has been added by querying the y2021_models table
pd.read_sql_query('select * from y2021_sales', con=engine).head()

,mfg,model,curr_month,month_prior_year,month_change,ytd_total,ytd_prior_year,ytd_change
0,Mazda,3,2463,3175,-22.43,32216,22634,42.33
1,Toyota,4Runner,13210,9860,33.98,94802,76598,23.77
2,Mazda,6,1435,1230,16.67,15529,10514,47.70
3,Volvo,60-Series,1153,1790,-35.59,9316,8975,3.80
4,Toyota,86 / FR-S,1,236,-99.58,1042,1816,-42.62


In [ ]:
# Use pandas to load models DataFrame into database
models_df.to_sql(name='y2021_models', con=engine, if_exists='replace', index=False)

In [ ]:
# Confirm data has been added by querying the y2021_models table
pd.read_sql_query('select * from y2021_models', con=engine).head()

In [ ]:
#Use pandas to load makes DataFrame into database
makes_df.to_sql(name='y2021_makes', con=engine, if_exists='replace', index=False)

In [ ]:
# Confirm data has been added by querying the y2021_makes table
pd.read_sql_query('select * from y2021_makes', con=engine).head()